# USGS 3DEP program
The U.S. Geological Survey’s (USGS) 3D Elevation Program (3DEP) is an ambitious effort to complete acquisition of nationwide lidar (IfSAR in AK) by 2023 to provide the first-ever national baseline of consistent high-resolution elevation data – both bare earth and 3D point clouds – collected in a timeframe of less than a decade.  USGS migrated these data into the cloud, led by Hobu, Inc. and the U.S Army Corps of Engineers (USACE) Cold Regions Research and Engineering Laboratory (CRREL) in collaboration with AWS Public Dataset Program.  As part of this effort the 3DEP data were standardized and written out in the entwine format.



# Entwine

- [Entwine](https://entwine.io/en/latest/index.html) is open source software from [HOBU, Inc.](https://hobu.co/) that organizes massive point cloud collections into streamable data services.  Entwine builds are completely lossless, so no points, metadata, or precision will be discarded even for terabyte-scale datasets.  The output format produced by entwine is the [Entwine Point Tile (EPT)](https://entwine.io/en/latest/entwine-point-tile.html), which is a simple and flexible octree-based storage format for point cloud data.  This format enables better performance when dealing with web map applications, or cloud computing with large point cloud datasets.

# EPT Data Accesss

- PDAL provides an [entwine reader](https://pdal.io/en/2.5.3/stages/readers.ept.html#readers-ept) that can easily read the USGS 3DEP data in EPT format. With a simple pipeline, it is fairly straightforward to subset the data from the AWS entwine bucket for a given dataset.  

- For example, to grab USGS 3DEP lidar data of the Statue of Liberty:

In [2]:
ls ./pipelines

AddGeoid.json		ExtractGround.json	StatueLiberty.json
Clip_ex.json		NoiseFilter.json	StatueLiberty_4326.json
CreateDSM.json		Part2_Exercise.json	StatueLiberty_Alt.json
CreateDTM.json		Part4_Exercise.json	TileLAZ.json
CreateGround.json	Reproject_Ex.json


In [3]:
more ./pipelines/StatueLiberty.json

{[?1h=
  "pipeline": [
        {
          "type": "readers.ept",
          "filename": "https://s3-us-west-2.amazonaws.com/usgs-lidar-public/NY_NewYorkCity/ept.json",
          "bounds": "([-8242666.7146411305, -8242531.114082908], [4966542.613156153, 4966683.155298185])"
        },
      "./data/StatueLiberty.laz"
  ]
}
>lines/StatueLiberty.json (END)


In [5]:
pdal pipeline ./pipelines/StatueLiberty.json

# Integrating 3DEP data into workflows

Based on the basic PDAL tutorials, we can apply additional processing stages to any of the datasets from the USGS 3DEP collection

The following pipeline does the following:
- Extract data over Devil's Tower Wyoming from the AWS enwtine bucket
- Reproject the data from the default Web Mercator to UTM Zone 13N
- Filter the data to just use ground-classified points
- Write out a ground-classified LAZ file
- Additionally write out a 2m DTM.

```
{
  "pipeline": [
      {
          "type": "readers.ept",
          "filename": "https://s3-us-west-2.amazonaws.com/usgs-lidar-public/WY_FEMA_East_B3_2019/ept.json",
          "bounds": "([-11657141., -11656509.],[5556970.,5557606. ])"
      },
      {
          "type":"filters.reprojection",
          "in_srs":"EPSG:3857+5703",
          "out_srs":"EPSG:32613+5703"
      },
      {
          "type": "filters.range",
          "limits": "Classification[2:2]"
      },
      {
          "type" : "writers.las",
          "filename": "./data/DevilsTower_Ground.laz",
          "compression": "laszip",
          "a_srs": "EPSG:32613+5703"
      },
      {
          "filename":"./data/DevilsTower_Ground.tif",
          "gdaldriver":"GTiff",
          "output_type":"min",
          "resolution":"2.0",
          "type": "writers.gdal"
      }	      
  ]
}
```

In [9]:
pdal pipeline ./pipelines/DevilsTower.json

In [7]:
gdalinfo ./data/DevilsTower_Ground.tif

Driver: GTiff/GeoTIFF
Files: ./data/DevilsTower_Ground.tif
Size is 227, 228
Coordinate System is:
COMPOUNDCRS["WGS 84 / UTM zone 13N + NAVD88 height",
    PROJCRS["WGS 84 / UTM zone 13N",
        BASEGEOGCRS["WGS 84",
            DATUM["World Geodetic System 1984",
                ELLIPSOID["WGS 84",6378137,298.257223563,
                    LENGTHUNIT["metre",1]]],
            PRIMEM["Greenwich",0,
                ANGLEUNIT["degree",0.0174532925199433]],
            ID["EPSG",4326]],
        CONVERSION["UTM zone 13N",
            METHOD["Transverse Mercator",
                ID["EPSG",9807]],
            PARAMETER["Latitude of natural origin",0,
                ANGLEUNIT["degree",0.0174532925199433],
                ID["EPSG",8801]],
            PARAMETER["Longitude of natural origin",-105,
                ANGLEUNIT["degree",0.0174532925199433],
                ID["EPSG",8802]],
            PARAMETER["Scale factor at natural origin",0.9996,
                SCALEUNIT["unity",1],
      

In [8]:
pdal info --metadata ./data/DevilsTower_Ground.laz

{
  "file_size": 2324613,
  "filename": "./data/DevilsTower_Ground.laz",
  "metadata":
  {
    "comp_spatialreference": "COMPD_CS[\"WGS 84 / UTM zone 13N + NAVD88 height\",PROJCS[\"WGS 84 / UTM zone 13N\",GEOGCS[\"WGS 84\",DATUM[\"WGS_1984\",SPHEROID[\"WGS 84\",6378137,298.257223563,AUTHORITY[\"EPSG\",\"7030\"]],AUTHORITY[\"EPSG\",\"6326\"]],PRIMEM[\"Greenwich\",0,AUTHORITY[\"EPSG\",\"8901\"]],UNIT[\"degree\",0.0174532925199433,AUTHORITY[\"EPSG\",\"9122\"]],AUTHORITY[\"EPSG\",\"4326\"]],PROJECTION[\"Transverse_Mercator\"],PARAMETER[\"latitude_of_origin\",0],PARAMETER[\"central_meridian\",-105],PARAMETER[\"scale_factor\",0.9996],PARAMETER[\"false_easting\",500000],PARAMETER[\"false_northing\",0],UNIT[\"metre\",1,AUTHORITY[\"EPSG\",\"9001\"]],AXIS[\"Easting\",EAST],AXIS[\"Northing\",NORTH],AUTHORITY[\"EPSG\",\"32613\"]],VERT_CS[\"NAVD88 height\",VERT_DATUM[\"North American Vertical Datum 1988\",2005,AUTHORITY[\"EPSG\",\"5103\"]],UNIT[\"metre\",1,AUTHORITY[\"EPSG\",\"9001\"]],AXIS[\"Gravi

    {
      "type": "VerticalCRS",
      "name": "NAVD88 height",
      "datum": {
        "type": "VerticalReferenceFrame",
        "name": "North American Vertical Datum 1988"
      },
      "coordinate_system": {
        "subtype": "vertical",
        "axis": [
          {
            "name": "Gravity-related height",
            "abbreviation": "",
            "direction": "up",
            "unit": "metre"
          }
        ]
      },
      "id": {
        "authority": "EPSG",
        "code": 5703
      }
    }
  ]
},
      "prettycompoundwkt": "COMPD_CS[\"WGS 84 / UTM zone 13N + NAVD88 height\",\n    PROJCS[\"WGS 84 / UTM zone 13N\",\n        GEOGCS[\"WGS 84\",\n            DATUM[\"WGS_1984\",\n                SPHEROID[\"WGS 84\",6378137,298.257223563,\n                    AUTHORITY[\"EPSG\",\"7030\"]],\n                AUTHORITY[\"EPSG\",\"6326\"]],\n            PRIMEM[\"Greenwich\",0,\n                AUTHORITY[\"EPSG\",\"8901\"]],\n            UNIT[\"degree\",0.01745329251994